In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.append("..")

In [4]:
from optimus import Optimus
op = Optimus()

In [5]:
op.spark

In [6]:
from pyspark.sql.types import StructType, StructField, StringType, BooleanType, IntegerType, ArrayType

df = op.create.df(
    [
        ("names", "str", True),
        ("height", "float", True),
        ("function", "str", True),
        ("rank", "int", True),
    ],
    [
        ("bumbl#ebéé  ", 17.5, "Espionage", 7),
        ("Optim'us", 28.0, "Leader", 10),
        ("ironhide&", 26.0, "Security", 7),
    ])
#.repartition(1)

df.table()

names 1 (string) nullable,height 2 (float) nullable,function 3 (string) nullable,rank 4 (int) nullable
bumbl#ebéé⸱⸱,17.5,Espionage,7
Optim'us,28.0,Leader,10
ironhide&,26.0,Security,7


In [7]:
df.table()

names 1 (string) nullable,height 2 (float) nullable,function 3 (string) nullable,rank 4 (int) nullable
bumbl#ebéé⸱⸱,17.5,Espionage,7
Optim'us,28.0,Leader,10
ironhide&,26.0,Security,7


In [28]:
df.rows.append(["Grimlock",80.0,"Commander",9])\
    .rows.select(df["rank"]>7)\
    .cols.select(["names","height","function"])\
    .cols.apply_expr("height", df["height"]-1)\
    .cols.drop("function")\
.show()

+--------+------+
|   names|height|
+--------+------+
|Optim'us|  27.0|
|Grimlock|  79.0|
+--------+------+



In [22]:
z = ['Grimlock', 80.0, 'Commander', 9]
[tuple(z)]

[('Grimlock', 80.0, 'Commander', 9)]

In [29]:
df = df\
    .rows.sort(["rank","height"])\
    .cols.lower(["names","function"])\
    .cols.remove_accents("names")\
    .cols.remove_special_chars("names")\
    .cols.trim("names")
df.show()

+---------+------+---------+----+
|    names|height| function|rank|
+---------+------+---------+----+
|  optimus|  28.0|   leader|  10|
| ironhide|  26.0| security|   7|
|bumblebee|  17.5|espionage|   7|
+---------+------+---------+----+



In [ ]:
print(df.cols.min("height"))
print(df.cols.percentile(['height', 'rank'], [0.05, 0.25, 0.5, 0.75, 0.95]))
print(df.cols.max("height"))
print(df.cols.median(["height","rank"]))
print(df.cols.range(["height","rank"]))
print(df.cols.std(["height","rank"]))

In [ ]:
df.show()

In [41]:
nest = df.cols.nest(["names", "function"], output_col = "new_col", separator=" ")
nest.show()

+---------+------+---------+----+-------------------+
|    names|height| function|rank|            new_col|
+---------+------+---------+----+-------------------+
|  optimus|  28.0|   leader|  10|     optimus leader|
| ironhide|  26.0| security|   7|  ironhide security|
|bumblebee|  17.5|espionage|   7|bumblebee espionage|
+---------+------+---------+----+-------------------+



In [36]:
nest_array = df.cols.nest(["height", "function"], output_col = "new_col", shape ="array")
nest_array.show()

+---------+------+---------+----+-----------------+
|    names|height| function|rank|          new_col|
+---------+------+---------+----+-----------------+
|  optimus|  28.0|   leader|  10|   [28.0, leader]|
| ironhide|  26.0| security|   7| [26.0, security]|
|bumblebee|  17.5|espionage|   7|[17.5, espionage]|
+---------+------+---------+----+-----------------+



In [43]:
nest_vector = df.cols.nest(["height", "rank"], output_col = "new_col", shape ="vector")
nest_vector.table()

names 1 (string) nullable,height 2 (float) nullable,function 3 (string) nullable,rank 4 (int) nullable,new_col 5 (vector) nullable
optimus,28.0,leader,10,"[10.0,28.0]"
ironhide,26.0,security,7,"[7.0,26.0]"
bumblebee,17.5,espionage,7,"[7.0,17.5]"


In [38]:
unnest = nest.cols.unnest("new_col", " ").cols.drop("new_col")
unnest.show()

+---------+------+---------+----+---------+---------+
|    names|height| function|rank|new_col_0|new_col_1|
+---------+------+---------+----+---------+---------+
|  optimus|  28.0|   leader|  10|  optimus|   leader|
| ironhide|  26.0| security|   7| ironhide| security|
|bumblebee|  17.5|espionage|   7|bumblebee|espionage|
+---------+------+---------+----+---------+---------+



In [45]:
unnest = nest_array.cols.unnest("new_col").cols.drop("new_col")
unnest.show()

+---------+------+---------+----+---------+---------+
|    names|height| function|rank|new_col_0|new_col_1|
+---------+------+---------+----+---------+---------+
|  optimus|  28.0|   leader|  10|     28.0|   leader|
| ironhide|  26.0| security|   7|     26.0| security|
|bumblebee|  17.5|espionage|   7|     17.5|espionage|
+---------+------+---------+----+---------+---------+



In [72]:
unnest = nest_vector.cols.unnest("new_col", index=0)
unnest.show()

+---------+------+---------+----+-----------+----+
|    names|height| function|rank|    new_col|  _6|
+---------+------+---------+----+-----------+----+
|  optimus|  28.0|   leader|  10|[10.0,28.0]|28.0|
| ironhide|  26.0| security|   7| [7.0,26.0]|26.0|
|bumblebee|  17.5|espionage|   7| [7.0,17.5]|17.5|
+---------+------+---------+----+-----------+----+



In [101]:
df.registerTempTable("autobots")
nest.sql("SELECT * FROM autobots").show()

+---------+------+---------+----+
|    names|height| function|rank|
+---------+------+---------+----+
|  optimus|  28.0|   leader|  10|
| ironhide|  26.0| security|   7|
|bumblebee|  17.5|espionage|   7|
+---------+------+---------+----+



In [112]:
from pyspark.sql import functions as F
def func(value, args):
    return value + args

df\
    .cols.apply("height", func, "int", 1)\
    .cols.apply_expr("rank", F.col("rank")+1)\
    .table()

names (string),height (int),function (string),rank (int)
optimus,29,leader,11
ironhide,27,security,8
bumblebee,18,espionage,8


In [117]:
from optimus.functions import abstract_udf as audf
def func(val, args):
    return val>8

df.rows.select(audf("rank", func, "bool")).show()

+-------+------+--------+----+
|  names|height|function|rank|
+-------+------+--------+----+
|optimus|  28.0|  leader|  10|
+-------+------+--------+----+



In [118]:
def func(val, args):
    return val+args[0]+args[1]

df.withColumn("height", audf ("height", func, "int", [1,2])).show()

+---------+------+---------+----+
|    names|height| function|rank|
+---------+------+---------+----+
|  optimus|    31|   leader|  10|
| ironhide|    29| security|   7|
|bumblebee|    20|espionage|   7|
+---------+------+---------+----+



In [119]:
def func(col_name, args):
    return F.col(col_name) + args

df.withColumn("height", audf ("height", func, "int", 2, "column_exp")).show()

+---------+------+---------+----+
|    names|height| function|rank|
+---------+------+---------+----+
|  optimus|  30.0|   leader|  10|
| ironhide|  28.0| security|   7|
|bumblebee|  19.5|espionage|   7|
+---------+------+---------+----+



In [42]:
df = df.melt(id_vars="names", value_vars=["height", "function","rank"])
df.table()

names (string),variable (string),value (string)
optimus,height,28.0
optimus,function,leader
optimus,rank,10
ironhide,height,26.0
ironhide,function,security
ironhide,rank,7
bumblebee,height,17.5
bumblebee,function,espionage
bumblebee,rank,7


In [43]:
df = df.pivot("names","variable","value")
df.table()

names (string),function (string),height (string),rank (string)
ironhide,security,26.0,7
optimus,leader,28.0,10
bumblebee,espionage,17.5,7


In [15]:
df.rows.append(["Grimlock",80.0,"Commander",9]).table()

names (string),height (float),function (string),rank (int)
bumbl#ebéé⸱⸱,17.5,Espionage,7
Optim'us,28.0,Leader,10
ironhide&,26.0,Security,7
Grimlock,80.0,Commander,9


In [45]:
df.rows.append(["Grimlock", "Commander","80", "9"]).table()

names (string),function (string),height (string),rank (string)
ironhide,security,26.0,7
optimus,leader,28.0,10
bumblebee,espionage,17.5,7
Grimlock,Commander,80,9


In [46]:
df.rows.sort("names").table()

names (string),function (string),height (string),rank (string)
optimus,leader,28.0,10
ironhide,security,26.0,7
bumblebee,espionage,17.5,7


In [47]:
df.rows.sort("names","asc").table()

names (string),function (string),height (string),rank (string)
bumblebee,espionage,17.5,7
ironhide,security,26.0,7
optimus,leader,28.0,10


In [49]:
df.cols.rename("names","name").table()

name (string),function (string),height (string),rank (string)
ironhide,security,26.0,7
optimus,leader,28.0,10
bumblebee,espionage,17.5,7


In [51]:
df.cols.rename([("names","name"),("function","task")])

DataFrame[name: string, task: string, height: string, rank: string]

In [66]:
df.cols.append("new_rank", df["rank"]+"1").show()

+---------+---------+------+----+--------+
|    names| function|height|rank|new_rank|
+---------+---------+------+----+--------+
| ironhide| security|  26.0|   7|     8.0|
|  optimus|   leader|  28.0|  10|    11.0|
|bumblebee|espionage|  17.5|   7|     8.0|
+---------+---------+------+----+--------+



In [62]:
print(type("12"+"123"))

<class 'str'>


In [73]:
df.cols.qcut("height","bins", 2).show()

+------------+------+---------+----+----+
|       names|height| function|rank|bins|
+------------+------+---------+----+----+
|bumbl#ebéé  |  17.5|Espionage|   7| 0.0|
|    Optim'us|  28.0|   Leader|  10| 1.0|
|   ironhide&|  26.0| Security|   7| 1.0|
+------------+------+---------+----+----+

